In [44]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from utilities import plot_confusion_matrix,plot_roc_auc,plot_classification_report
from utilities import grid_search_simple,graph_result,get_classes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [45]:
full_df = pd.read_csv("..\..\data\indeed.pre_processing.csv")

In [46]:
df = full_df.copy()
cols_to_drop = ["Unnamed: 0.1","adresse","date_de_publication","description","localisation","nom_entreprise","salaire",
        "titre","type_de_contrat","url","niveau_etude","type_de_cursus","type_contrat_description","grande_categorie", "type_contrat_titre"
               ,'salaire_min','salaire_max','3 mois renouvelable_desc','3mois renouvelable_desc']
df.drop(cols_to_drop, axis=1, inplace=True)

In [47]:
y = df['salaire_moyen'][pd.notnull(df['salaire_moyen'])]
X = df.drop(columns = ['salaire_moyen',"_id"])
X = X[pd.notnull(df['salaire_moyen'])]

In [48]:
y_class, dic_classe_labels,dic_min_max_mean = get_classes(y)

In [49]:
svm = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.25, random_state=0)

In [51]:
svm.fit(X_train,y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
y_pred_reglog = svm.predict(X_test)

In [53]:
accuracy_score(y_test, y_pred_reglog)

0.803680981595092

In [54]:
df_predicted = df.drop(columns = ['salaire_moyen'])
df_predicted.reset_index(inplace=True, drop=True)
X_df_predicted = df_predicted.drop(columns = ['_id'])
X_df_predicted.reset_index(inplace=True, drop=True)

In [55]:
#%%time
y_pred_w_s = svm.predict(X_without_s)

In [56]:
salaire_moyen = []
salaire_min = []
salaire_max = []
for i in y_pred_w_s:
    salaire_min.append(dic_min_max_mean[i][0])
    salaire_moyen.append(dic_min_max_mean[i][1])
    salaire_max.append(dic_min_max_mean[i][2])

In [57]:
df_predicted['salaire_moyen'] = pd.DataFrame(salaire_moyen)
df_predicted['salaire_min'] = pd.DataFrame(salaire_min)
df_predicted['salaire_max'] = pd.DataFrame(salaire_max)

In [58]:
cols =["adresse","type_de_contrat","niveau_etude", "localisation","type_de_cursus"]
df_predicted[cols] = full_df[pd.isna(full_df["salaire_moyen"])][cols]

In [59]:
cols.append("salaire_moyen")
cols.append("salaire_min")
cols.append("salaire_max")
df_predicted = df_predicted[cols]
df_predicted

,adresse,type_de_contrat,niveau_etude,localisation,type_de_cursus,salaire_moyen,salaire_min,salaire_max
0,Toulouse (31),NaN,ingénieur,Toulouse,NaN,37000.0,2700,47000
1,Toulouse (31),CDI,NaN,Toulouse,NaN,37000.0,2700,47000
2,Labège (31),CDI,ingénieur,Toulouse,NaN,66000.0,4700,85000
3,Toulouse (31),CDI,ingénieur,Toulouse,autodidacte,37000.0,2700,47000
4,NaN,NaN,NaN,NaN,NaN,37000.0,2700,47000
5,Toulouse (31),NaN,ingénieur,Toulouse,NaN,37000.0,2700,47000
6,Toulouse (31),NaN,ingénieur,Toulouse,NaN,37000.0,2700,47000
7,Toulouse (31),NaN,ingénieur,Toulouse,NaN,37000.0,2700,47000
8,NaN,NaN,NaN,NaN,NaN,13500.0,0,27000
9,Paris (75),NaN,NaN,Paris,NaN,66000.0,4700,85000


In [60]:
df_predicted.to_csv("..\..\data\indeed.predicted.csv")